In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random 
import re
from scipy import spatial
%matplotlib inline
import math 
from collections import Counter 
import nltk
from operator import itemgetter 
from scipy.spatial import distance
import sys
from tqdm import tqdm
import itertools
from itertools import permutations 
import pylab
import tqdm 
import time as time1
from textblob import TextBlob
from statistics import mean
import pickle


In [ ]:
#load paris rating into dataframe
rating = pd.read_csv("paris_rating.csv" , names=["id" , "Score"]) 

POI=rating["id"].tolist()
rating.head()


In [ ]:
#load paris tt into dataframe
Time = pd.read_csv("paris_tt.csv") 
Time.head()

In [ ]:
#load paris visittime into dataframe
visit_time = pd.read_csv("paris_visittime.csv" , names=["id" ,"Type", "time"]) 
visit_time.head()


In [ ]:
#load paris types into dataframe
Type = pd.read_csv("paris_types.csv", names=["id" , "Type"]) 
Type.head()

In [ ]:
#load paris output into dataframe
Distances = pd.read_csv("paris_output.csv") 
Distances.head()

In [ ]:
#load paris datawithtypes into dataframe
goal=pd.read_csv("paris_datawithtypes.csv", names=["POIs" , "Types"]) 
goal.head()

In [ ]:
#Generate a complete graph with the POIs as nodes

edges=[]
for i in POI:
    poi=POI[:]
    poi.remove(i)
    for j in poi:
        edges.append((i,j))

for poi in POI:
    edges.append(('initial' , poi))

G = nx.Graph()
G.add_edges_from(edges) 
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G,pos, cmap=plt.get_cmap('jet'),node_size = 1000,node_color="lightblue")
nx.draw_networkx_edges(G,pos,edge_color='b', edge_cmap=plt.cm.Blues,arrows=True,arrowstyle="->",arrowsize=10)
nx.draw_networkx_labels(G,pos)

plt.show()

In [ ]:
# get cell value in dataframe

def cellValue(df,rowName, columnName):
    
    index=df[df["id"]==rowName].index[0]

    return df.iloc[index][columnName]

In [ ]:
#Calculate the visitation time of a sequence of POIs

def visitTime(seq):
    
    total=0
    for i in seq:
        
        total+=int(cellValue(visit_time ,i, "time"))
        
    return total
        

In [ ]:
#Calculate the commute time of a sequence of POIs

def commuteTime(seq):
    
    total=0

    for i in range(len(seq)-1):

        total+=int(cellValue(Time ,seq[i], seq[i+1]))
        
    return total/60

In [ ]:
def gap(List, POI):
    
    if cellValue(Type, POI, "Type") in List:

        if (len(List)- get_index_positions(List,cellValue(Type, POI, "Type"))[0]) < 3:
            return 0 
        else:
            return 1
        
    else:
        return 1


In [ ]:
nodes={}
for i,node in enumerate(G.nodes()):
    nodes[i]=node

In [ ]:
def getKeysByValue(dictOfElements, valueToFind):
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            key=item[0]
    return  key

In [ ]:
#Function to find key based on value in node dictionary
def findKey(poi):
    return list(nodes.keys())[list(nodes.values()).index(poi)]


In [ ]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

In [ ]:
''' Returns the maximum number of consecutive numbers that occured in a list '''

def consecutive(list_of_element,element):
    
    list_of_index=get_index_positions(list_of_element, element)
    
    occurance=[]
    count=1
    i=0
    if (len( list_of_index)>=2):
        while i < (len(list_of_index)-1):
            if (list_of_index[i+1] == list_of_index[i]+1):
                count+=1
                i+=1
            else:
                occurance.append(count)
                i+=1
                count=1
        occurance.append(count)

        maximum=max(occurance)
        length=len(list_of_element)
        sum_list=sum(occurance)
        occur=(sum_list * maximum) / length
    elif (list_of_index==[]):
        occur=0
    else:
        occur=list_of_element[list_of_index[0]]

    return occur

In [ ]:
#Class that defines the Enviornment
#Our enviornment follows a graph based model of a data set

class Env:
    def __init__(self,input_graph,goal_vect,startState):
        
        self.input_graph = input_graph #NetworkX graph
        self.goal_vect = goal_vect #Goal Vector/Features
        self.startState = startState #Integer representing the starting state 
   

  
    def step(self,state1,action1,state1Vect, stateList,delta,beta, POI_vector,  time): # to take the action 
        
        done = False


        #Take a specified action and get new state 

        state1Vect.append(cellValue(Type , action1[1] , "Type"))
        state2Vect = state1Vect[:]
        

            
        totalTime=visitTime(POI_vector)+commuteTime(POI_vector)
        if (totalTime >= time): #Have same num of courses as the goal vector, so terminate
            done = True

        #Calculate reward of the given move
        rewardList=[]
        
        
        
        for i in range(len(self.goal_vect)):

            if len(state2Vect)<=len(self.goal_vect[i]):
                distance=state2Vect[len(state2Vect)-1] == self.goal_vect[i][len(state2Vect)-1]

                if distance==True:
                    stateList[i].append(1)
                else:
                    stateList[i].append(0)

                rewardList.append(consecutive(stateList[i],1))

            else:
                continue



        rewardTotal= (delta * mean(rewardList) + beta * int(cellValue(rating , action1[1] , "Score")))


        state2 = action1[1]
        # Return second state, reward,done flag
        return state2,state2Vect,rewardTotal,stateList,done

 


    def reset (self):
    #This will reset to initial state 
    # We invoke this on each episode 
        return self.startState



In [ ]:
#Define function to choose action given a state, as per policy of our paper
#Policy is applied to the seed state on each iteration

def choose_action(state,stateUTSoFar,goalVector,possible_states,stateVector, stateList,POI_vector,delta,beta, time, dis):
  

    possible_actions={}
   # Get list of possible actions 
    for action in list(G.edges()):
    
        if (action[0] == state and (action[1] in possible_states)== True):
            

            a=cellValue(Distances , state , action[1])

            
            if (a <= dis): # This is distance threshold to choose action
                possible_actions[action] = cellValue(Type, action[1] , 'Type')
        else:
            continue

   
    utility={}
    for action in possible_actions:

        state_vect=stateVector[:]
        POIVector=POI_vector[:]
        POIVector.append(action[1])
        state_vect.append(possible_actions[action])
        
        
        totalTime=visitTime(POIVector)+commuteTime(POIVector)
        if (totalTime < time):

            List=[]

            state_list=copy.deepcopy(stateList)
            for i in range(len(goalVector)):
                
                if len(state_vect)<=len(goalVector[i]):
                    
                    distance=state_vect[len(state_vect)-1] == goalVector[i][len(state_vect)-1]

                    if distance==True:
                        state_list[i].append(1)
                    else:
                        state_list[i].append(0)

                    List.append(consecutive(state_list[i],1))
                    
                else:
                    continue
                    print(False)
   
            currStateUT=0
            currStateUT= (delta * mean(List) + beta * int(cellValue(rating , action[1] , "Score"))) * gap(stateVector , action[1])
            sess_utility  = currStateUT + stateUTSoFar 
            utility[action]=sess_utility
        
        else:
            break


  
    candidate=[]
    if (utility!={}):
        maximum=max(utility.values())
        for i in utility:
            if (utility[i]==maximum):
                candidate.append(i)
        chosen_action = random.choice(candidate)
        stateUTSoFar=utility[chosen_action]

    else:
        chosen_action=()
 

    return chosen_action , stateUTSoFar 


In [ ]:
#Choose First action

def choose_action1(state,stateUTSoFar,goalVector,possible_states,stateVector, stateList,POI_vector,delta,beta, time, dis):
  


    state1=random.choice(POI)
    action=('initial' , state1)
    state_vect=stateVector[:]
    POIVector=POI_vector[:]
    POIVector.append(action[1])
    state_vect.append(cellValue(Type, action[1] , 'Type'))
        

    List=[]

    state_list=copy.deepcopy(stateList)
    for i in range(len(goalVector)):

        if len(state_vect)<=len(goalVector[i]):

            distance=state_vect[len(state_vect)-1] == goalVector[i][len(state_vect)-1]

            if distance==True:
                state_list[i].append(1)
            else:
                state_list[i].append(0)

            List.append(consecutive(state_list[i],1))

        else:
            continue



    currStateUT= (delta * mean(List) + beta * int(cellValue(rating , action[1] , "Score"))) 
    stateUTSoFar  = currStateUT + stateUTSoFar 


    return action , stateUTSoFar 

In [ ]:
def getIdealVector(goal , time, dis):
    
    idealVect_type=[]
    idealVect=[]
    for i in range(len(goal.index)):

        if (len(goal.iloc[i]["Types"].split(','))>1):

            idealVect_type.append(goal.iloc[i]["Types"].split(','))
            idealVect.append(goal.iloc[i]["POIs"].split(','))
            
            
    index=[]       
    for i in range(len(idealVect)):
        if (commuteTime(idealVect[i])+visitTime(idealVect[i])> time):
            index.append(i)
            
            
    for i in range(len(idealVect)):
        for j in range(len(idealVect[i])-1):
            if cellValue(Distances , idealVect[i][j] , idealVect[i][j+1]) <= dis:
                continue
            else:
                index.append(i)
    x=[]

    for j in set(index):
        x.append(idealVect_type[j])

        
    for k in x:
        idealVect_type.remove(k)
    
    
    return idealVect_type
            
            

In [ ]:
#Initialize parameters (Set as per preferences)

time=6
dis=5


#Initializing the Q-matrix 
numOfStates = G.number_of_nodes() 


idealVect=getIdealVector(goal, time, dis)       
print(len(idealVect))
 

startState='initial'

env = Env(G, idealVect,startState)

In [ ]:
#Do learning. 
import copy
itinerary=[]

states=POI[:]
reward = 0
#ideal composition weight
delta=0.6
beta=0.4

stateUTSoFar = 0 
state1 = env.reset()

POI_vector=[]
state1Vect = []
stateList1=[[]]*len(idealVect)
action1,stateUTSoFar= choose_action1(state1,stateUTSoFar,idealVect, states,state1Vect,stateList1,POI_vector,delta,beta, time, dis)

POI_vector.append(action1[1])


while True: 

    #Getting the next state 

    state2, state2Vect ,reward, stateList2, done = env.step(state1,action1,state1Vect,stateList1,delta,beta, POI_vector, time)
    states.remove(state2)


    #Choosing the next action 
    stateList3=copy.deepcopy(stateList2)
    action2,stateUTSoFar= choose_action(state2,stateUTSoFar,idealVect, states, state2Vect, stateList3,POI_vector,delta,beta, time, dis) 



    if (action2==()):
        print(POI_vector)
        break
    else:
        POI_vector.append(action2[1])
        itinerary.append(POI_vector)
        
        
        
    state1 = state2 
    action1 = action2 
    state1Vect = state2Vect
    stateList1=stateList2

    #If at the end of learning process, start another session
    if (done==True or states==[]): 
        break


In [ ]:
Sequence=POI_vector

In [ ]:
#total time of the result

totalTime=visitTime(Sequence)+commuteTime(Sequence)
totalTime

In [ ]:
#Type of POIs in the result as well as the score of recommended itinerary

poi_type=[]
score=[]
for poi in Sequence:
    poi_type.append(cellValue(Type, poi, 'Type'))
    score.append(cellValue(rating, poi, 'Score'))
    
print(score)
print(poi_type)
print(mean(score))